In [1]:
import sys
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 3 -p
%matplotlib inline

## imports

In [3]:
import torch
from torch import nn

In [4]:
import yolov1
from yolov1.config import parse_config

In [5]:
from yolov1.data.dataset import YOLODataset
from yolov1.utils.vis import draw_boxes, draw_box

## config

In [72]:
config_path = "../yolov1/src/yolov1/configs/default.yaml"

Reloading 'yolov1.data.dataset'.
Reloading 'yolov1.config'.


In [73]:
config = parse_config(config_file=config_path)

config

YOLOConfig(data=DataConfig(root='/home/ajkdrag/workspace/projects/architectures-impl-pytorch/.data', train='train', val='valid', names=['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck']), training=TrainingConfig(batch_size=8, num_workers=4, learning_rate=0.001, epochs=50), model=ModelConfig(backbone='resnet18', pretrained=True, backbone_output_channels=512, detector_hidden_sz=1024, input_size=(224, 224), S=7, B=1, nc=5))

## dataset

In [74]:
train_ds = YOLODataset(config, mode="train")

2024-05-04T17:01:41.686380Z [info     ] Loaded 878 samples             func_name=__init__ lineno=37


In [75]:
val_ds = YOLODataset(config, mode="val")

2024-05-04T17:01:41.816393Z [info     ] Loaded 250 samples             func_name=__init__ lineno=37


In [76]:
val_ds.encode = True

In [77]:
val_ds[0][1]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.000

In [78]:
# draw_boxes(val_ds[3][0], val_ds[3][1])

## dataloader

In [79]:
train_dataloader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=config.training.batch_size,
        shuffle=True,
        num_workers=config.training.num_workers,
    )
val_dataloader = torch.utils.data.DataLoader(
    val_ds,
    batch_size=config.training.batch_size,
    shuffle=False,
    num_workers=config.training.num_workers
)

## arch

In [80]:
from yolov1.models.arch import YOLOv1

In [81]:
config.model

ModelConfig(backbone='resnet18', pretrained=True, backbone_output_channels=512, detector_hidden_sz=1024, input_size=(224, 224), S=7, B=1, nc=5)

In [82]:
model = YOLOv1(config.model)

Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
[timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


## train

In [83]:
from yolov1.train import main

In [84]:
main(config)

2024-05-04T17:01:48.687387Z [info     ] Loaded 878 samples             func_name=__init__ lineno=37
Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
[timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


/home/ajkdrag/workspace/projects/architectures-impl-pytorch/.venv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


KeyboardInterrupt: 

## persist

## infer